In [1]:
import re
import math
import random
import tkinter as tk
from tkinter import filedialog

In [2]:
class Node:
    radius = 10
    small_radius = 3
    
    def __init__(self, network, x, y, text):
        self.index = -1
        self.network = network
        self.x = x
        self.y = y
        self.text = text
        self.links = []
        
        self.network.add_node(self)
        
    def __str__(self):
        return f"[{self.text}]"
    
    def add_link(self, link):
        self.links.append(link)
        
    def draw(self, canvas, draw_labels=True):
        radius = self.radius if draw_labels else self.small_radius
        canvas.create_oval(
            self.x - radius, self.y - radius,
            self.x + radius, self.y + radius,
            fill='white',
            outline='black'
        )
        if draw_labels:
            canvas.create_text(
                self.x, self.y,
                text=self.text,
                fill='blue'
            )

class Link:
    def __init__(self, network, from_node, to_node, cost):
        self.network = network
        self.from_node = from_node
        self.to_node = to_node
        self.cost = cost
    
        self.network.add_link(self)
        self.from_node.add_link(self)
        
    def __str__(self):
        return f"[{self.from_node.text}] --> [{self.to_node.text}] ({self.cost})"
    
    def draw(self, canvas):
        # Draw link one way only, avoid having to draw bidirectional links twice
        if self.from_node.index < self.to_node.index:
            canvas.create_line(
                self.from_node.x,
                self.from_node.y,
                self.to_node.x,
                self.to_node.y,
                fill='green'
            )
    
    def draw_label(self, canvas):
        # Draw a label along this link.
        dx = self.to_node.x - self.from_node.x
        dy = self.to_node.y - self.from_node.y
        
        angle_radians = math.atan2(dx, dy)
        angle_degrees = angle_radians * 180 / math.pi
        # Subtract 90 degrees so that text will run parallel to the link
        angle_degrees -= 90
        
        # Find the point one-third of the way along link, using weighted average
        x = 0.67 * self.from_node.x + 0.33 * self.to_node.x
        y = 0.67 * self.from_node.y + 0.33 * self.to_node.y
        
        # Erase part of the link
        radius = 10
        canvas.create_oval(
            x - radius, y - radius,
            x + radius, y + radius,
            fill='white',
            outline=''
        )
        
        canvas.create_text(
            x, y,
            text=str(self.cost),
            angle=angle_degrees
        )

class Network:
    def __init__(self):
        self.clear()
    
    def clear(self):
        self.nodes = []
        self.links = []
        
    def add_node(self, node: Node):
        node.index = len(self.nodes)
        self.nodes.append(node)
        
    def add_link(self, link: Link):
        self.links.append(link)
        
    def to_string(self):
        return (
            "{} # Num nodes.".format(len(self.nodes)) +
            "\n{} # Num links.".format(len(self.links)) +
            "\n# Nodes." +
            "\n{}".format('\n'.join([f"{n.x},{n.y},{n.text}" for n in self.nodes])) +
            "\n# Links." +
            "\n{}".format('\n'.join([f"{l.from_node.index},{l.to_node.index},{l.cost}" for l in self.links]))
        )
    
    def save_into_file(self, file_name):
        open(file_name, 'w').write(self.to_string())
    
    @staticmethod
    def read_next_line(file_handle):
        while line := file_handle.readline():
            pattern = re.compile('([^#]*)(#.*)?$')
            matcher = pattern.match(line)
            line = matcher.group(1).strip() if matcher else None
            if line:
                return line 
        return None
    
    def load_from_file(self, file_name):
        self.clear()
        
        with open(file_name, 'r') as file_handle:
            num_nodes = int(self.read_next_line(file_handle))
            num_links = int(self.read_next_line(file_handle))
            # get nodes
            while num_nodes > 0 and (line := self.read_next_line(file_handle)):
                num_nodes -= 1
                if m := re.match(r'(\d+),(\d+),(.*)', line):
                    x = int(m.group(1))
                    y = int(m.group(2))
                    text = m.group(3)
                    Node(self, x, y, text)
            # get links
            while num_links > 0 and (line := self.read_next_line(file_handle)):
                num_links -= 1
                if m := re.match(r'(\d+),(\d+),(\d+)', line):
                    from_node_idx = int(m.group(1))
                    to_node_idx = int(m.group(2))
                    cost = int(m.group(3))
                    Link(self, self.nodes[from_node_idx], self.nodes[to_node_idx], cost)

    def draw(self, canvas):
        draw_labels = True if len(self.nodes) < 100 else False
        
        for l in self.links:
            l.draw(canvas)
        if draw_labels:
            for l in self.links:
                l.draw_label(canvas)
            
        for n in self.nodes:
            n.draw(canvas, draw_labels)


In [3]:
class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None
        
        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('draw_network')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('600x400')
        
        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.menubar, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_network,
                                   accelerator='Ctrl+O')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)
        
        # Build the window surface.
        self.canvas = tk.Canvas(self.window, bg='white', borderwidth=2, relief=tk.SUNKEN)
        self.canvas.pack(padx=10, pady=(0, 10), side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        self.window.bind('<Control-o>', self.ctrl_o_pressed)
        
        # Display the window.
        self.window.focus_force()
        self.window.mainloop()
        
    def kill_callback(self):
        self.window.destroy()
        
    def ctrl_o_pressed(self, event):
        self.open_network()
    def open_network(self):
        if file_name := filedialog.askopenfilename(initialdir='./'):
            self.network = Network()
            self.network.load_from_file(file_name)
            self.draw_network()
    
    def draw_network(self):
        # Clear any previous drawing.
        self.canvas.delete()
        
        if self.network:
            self.network.draw(self.canvas)


In [4]:
App()


In [5]:
def build_grid_network(file_name, width, height, num_rows, num_cols):
    network = Network()
    node_id = 0
    margin = 10
    x = 0 + margin + Node.radius
    y = 0 + margin + Node.radius

    x_space = int((width - 2 * margin) / num_cols)
    y_space = int((height - 2 * margin) / num_rows)

    # Nodes will be stored in the network's instance in a list
    for r in range(0, num_rows):
        for c in range(0, num_cols):
            Node(network, x, y, node_id)
            node_id += 1
            x += x_space + Node.radius
            
        x = 0 + margin + Node.radius
        y += y_space + Node.radius

    # When creating links refer to network's nodes list
    for r in range(0, num_rows):
        for c in range(0, num_cols):
            node = network.nodes[r*num_cols + c]
            
            adjacent_top = (r-1)*num_cols + c if r > 0 else None
            if adjacent_top is not None:
                to_node = network.nodes[adjacent_top]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_right = r*num_cols + c+1 if c < num_cols-1 else None
            if adjacent_right is not None:
                to_node = network.nodes[adjacent_right]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_bottom = (r+1)*num_cols + c if r < num_rows-1 else None
            if adjacent_bottom is not None:
                to_node = network.nodes[adjacent_bottom]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_left = r*num_cols + c-1 if c > 0 else None
            if adjacent_left is not None:
                to_node = network.nodes[adjacent_left]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
    
    network.save_into_file(file_name)

build_grid_network('grid_network.net', 600, 400, 10, 15)
